In [2]:
!pip install boto3

     -------------------------------------- 132.5/132.5 kB 3.8 MB/s eta 0:00:00
  Using cached botocore-1.27.89-py3-none-any.whl (9.2 MB)
     ---------------------------------------- 79.6/79.6 kB 4.6 MB/s eta 0:00:00


In [3]:
import boto3
import pandas as pd

In [4]:
# DO NOT PUSH CREDENTIALS TO REPO
s3 = boto3.client(
    service_name='s3',
    region_name='us-west-2',
    aws_access_key_id='',
    aws_secret_access_key=''
)

In [12]:
# %time
# response = s3.get_object(Bucket='reddit-title-generation', Key="dataset-train.csv")

# status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

# if status == 200:
#     print(f"Successful S3 get_object response. Status - {status}")
#     train = pd.read_csv(response.get("Body"))
#     print(train.head())
# else:
#     print(f"Unsuccessful S3 get_object response. Status - {status}")

CPU times: total: 0 ns
Wall time: 0 ns
Successful S3 get_object response. Status - 200
   Unnamed: 0           author  \
0     2099088  UghImRegistered   
1     2893884  PossibleLesbian   
2     2237635        [deleted]   
3      498777          Naztash   
4     1337130        BurChaBow   

                                                body  \
0  &gt; “I have friends with the same degree as m...   
1  Just a bit of background: I grew up Catholic. ...   
2  I myself enjoy approaching an attractive young...   
3  You do realize that the contract probably has ...   
4  [](/dashiewilliamisboredofnamingemotes)\n\nI g...   

                                      normalizedBody       subreddit  \
0  > “I have friends with the same degree as me, ...        politics   
1  Just a bit of background: I grew up Catholic. ...  actuallesbians   
2  I myself enjoy approaching an attractive young...       AskReddit   
3  You do realize that the contract probably has ...         TopGear   
4  \n I got

In [8]:
# Querying CSV for relevant results
resp = s3.select_object_content(
    Bucket='reddit-title-generation',
    Key='dataset-train.csv',
    ExpressionType='SQL',
    # Need to use positional headers in query
    Expression="SELECT * FROM s3object s WHERE CHAR_LENGTH(s._2) < 100 LIMIT 10000",
    InputSerialization = {'CSV': {"FileHeaderInfo": "NONE", 'AllowQuotedRecordDelimiter':True}, 'CompressionType': 'NONE'},
    OutputSerialization = {'CSV': {}},
)

records = []
for event in resp['Payload']:
    if 'Records' in event:
        # records.append(event['Records']['Payload'].decode('utf-8'))
        records.append(event['Records']['Payload'])  
        
file_str = ''.join(req.decode('utf-8') for req in records)

EventStreamError: An error occurred (InternalError) when calling the SelectObjectContent operation: We encountered an internal error. Please try again.

Maximum length per row in data is 1MB, which I believe is causing issues.

In [ ]:
from io import StringIO
train = pd.read_csv(StringIO(file_str), header=0)
train.head()

# Imports

In [70]:
from fastai.text.all import *
from transformers import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

#Select part of data we want to keep
train_texts = train[['content','summary']]

#Clean text

## BART

In [17]:
pretrained_model_name = "facebook/bart-large-cnn"
hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name, 
                                                                  model_cls=BartForConditionalGeneration)

# Create mini-batch and define parameters
hf_batch_tfm = Seq2SeqBatchTokenizeTransform(hf_arch, hf_config, hf_tokenizer, hf_model, 
    task='summarization')

# Simple preprocessing
preprocessor = SummarizationPreprocessor(
    hf_tokenizer,
    text_attr='content',
    target_text_attr='summary',
    max_input_tok_length=256,
    max_target_tok_length=130,
    min_summary_char_length=30,
)

preprocessed_train = preprocessor.process_df(train_texts)


# Prepare data for training
blocks = (Seq2SeqTextBlock(batch_tokenize_tfm=hf_batch_tfm), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader('content'), get_y=ColReader('summary'), splitter=RandomSplitter())
dls = dblock.dataloaders(preprocessed_train, bs = 2)

C:\Users\ikim1\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
C:\Users\ikim1\anaconda3\lib\site-packages\blurr\text\data\seq2seq\summarization.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(self._process_df_batch(batch_df))
C:\Users\ikim1\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your i

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


C:\Users\ikim1\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [18]:
#Define performance metrics
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]}}

#Model
model = BaseModelWrapper(hf_model)
learn_cbs = [BaseModelCallback]
fit_cbs = [Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

#Specify training
learn = Learner(dls, model,
                opt_func=ranger,loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,splitter=partial(blurr_seq2seq_splitter, arch=hf_arch)).to_fp16()

#Create optimizer with default hyper-parameters
learn.create_opt() 
learn.freeze()

#Training
learn.fit_one_cycle(3, lr_max=3e-5, cbs=fit_cbs)

#Exporting model
learn.save('reddit_summary.pkl')

C:\Users\ikim1\anaconda3\lib\site-packages\torch\amp\autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\ikim1\anaconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,bertscore_precision,bertscore_recall,bertscore_f1,time
0,3.342315,3.172942,0.202566,0.047688,0.139865,0.653562,0.695616,0.673121,2:57:38
1,2.924973,3.115966,0.209516,0.053569,0.148705,0.654510,0.695004,0.673398,2:21:10
2,2.240080,3.207419,0.211030,0.053737,0.148333,0.655003,0.697316,0.674661,2:36:40


C:\Users\ikim1\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
C:\Users\ikim1\anaconda3\lib\site-packages\transformers\generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [22]:
#Exporting model
learn.save('reddit_summary.pkl')

Path('models/reddit_summary.pkl.pth')

In [28]:
# #Loading model
# learn.load('reddit_summary.pkl')

In [39]:
outputs = learn.blurr_generate(train_texts.iloc[0].content, early_stopping=False, num_return_sequences=1, \
                               min_length=30, max_length=50)

for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')

=== Prediction 1 ===
{'generated_texts': " co-op programs can make you more marketable, not less marketable.  It's not just about luck, it's about networking, experience and professionalism.  If you want to get out of your rut,"}



In [35]:
train_texts.iloc[0].content

"I have friends with the same degree as me, from a worse school, but because of who they knew or when they happened to graduate, they’re in much better jobs,” said Kyle Bishop, 23, a 2009 graduate of the University of Pittsburgh who has spent the last two years waiting tables, delivering beer, working at a bookstore and entering data. “It’s more about luck than anything else.” \n It's not  just  about luck, Kyle.  It's also about networking, experience and professionalism. \n This is why you should, if you have the opportunity, enrol in a co-op program.  I graduated with 2 full years of real-world work experience in my field and had a job lined up before I graduated.  In fact, I still don't technically have my degree because I was a lazy student and had to take some part time courses while I worked full time.  But that hasn't stopped me from already being hired for two jobs that pay 65k+ during the tail end of a recession.  At the end of the day they hire me because I can prove my effe

## T5

In [71]:
pretrained_model_name = "t5-base"
hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name, 
                                                                  model_cls=T5ForConditionalGeneration)

# Create mini-batch and define parameters
hf_batch_tfm = Seq2SeqBatchTokenizeTransform(hf_arch, hf_config, hf_tokenizer, hf_model, 
    task='summarization')

# Simple preprocessing
preprocessor = SummarizationPreprocessor(
    hf_tokenizer,
    text_attr='content',
    target_text_attr='summary',
    max_input_tok_length=256,
    max_target_tok_length=130,
    min_summary_char_length=30,
)

preprocessed_train = preprocessor.process_df(train_texts)


# Prepare data for training
blocks = (Seq2SeqTextBlock(batch_tokenize_tfm=hf_batch_tfm), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader('content'), get_y=ColReader('summary'), splitter=RandomSplitter())
dls = dblock.dataloaders(preprocessed_train, bs = 2)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

C:\Users\ikim1\anaconda3\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

C:\Users\ikim1\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
C:\Users\ikim1\anaconda3\lib\site-packages\blurr\text\data\seq2seq\summarization.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(self._process_df_batch(batch_df))
C:\Users\ikim1\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your i

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


C:\Users\ikim1\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [72]:
#Define performance metrics
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]}}

#Model
model = BaseModelWrapper(hf_model)
learn_cbs = [BaseModelCallback]
fit_cbs = [Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

#Specify training
learn = Learner(dls, model,
                opt_func=ranger,loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,splitter=partial(blurr_seq2seq_splitter, arch=hf_arch)).to_fp16()

#Create optimizer with default hyper-parameters
learn.create_opt() 
learn.freeze()

In [22]:
#Exporting model
learn.save('reddit_summary.pkl')

Path('models/reddit_summary.pkl.pth')

In [73]:
#Training
learn.fit_one_cycle(1, lr_max=3e-5, cbs=fit_cbs)

#Exporting model
learn.save('t5_reddit_summary.pkl')

C:\Users\ikim1\anaconda3\lib\site-packages\torch\amp\autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\ikim1\anaconda3\lib\site-packages\torch\cuda\amp\grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,bertscore_precision,bertscore_recall,bertscore_f1,time
0,3.803366,3.810098,0.126422,0.020688,0.103389,0.654869,0.623079,0.637797,1:09:34


C:\Users\ikim1\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
C:\Users\ikim1\anaconda3\lib\site-packages\transformers\generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Path('models/t5_reddit_summary.pkl.pth')

In [88]:
outputs = learn.blurr_generate(train_texts.iloc[0].content, early_stopping=True, \
                               # Exploring decoding strategies
                               do_sample=True, top_k=50,top_p=0.95,\
                               min_length=30, max_length=50, num_return_sequences=3)

for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')

=== Prediction 1 ===
{'generated_texts': ['I graduated with two full years of real-world work experience in my field . But the things I learned are a lot harder for me to overcome .', 'the cost of coop is high and the benefits are huge. What are some co-op ways to save money on college? coop is much more about people having a second career and coop is about professional development as well as', 'the result is more potential employer value than any other degree you could have earned. “I work in tech, but my job is not in business.”']}



In [77]:
train_texts.iloc[0].content

"I have friends with the same degree as me, from a worse school, but because of who they knew or when they happened to graduate, they’re in much better jobs,” said Kyle Bishop, 23, a 2009 graduate of the University of Pittsburgh who has spent the last two years waiting tables, delivering beer, working at a bookstore and entering data. “It’s more about luck than anything else.” \n It's not  just  about luck, Kyle.  It's also about networking, experience and professionalism. \n This is why you should, if you have the opportunity, enrol in a co-op program.  I graduated with 2 full years of real-world work experience in my field and had a job lined up before I graduated.  In fact, I still don't technically have my degree because I was a lazy student and had to take some part time courses while I worked full time.  But that hasn't stopped me from already being hired for two jobs that pay 65k+ during the tail end of a recession.  At the end of the day they hire me because I can prove my effe